<a href="https://colab.research.google.com/github/NRJaggers/Applied-Stochastic-Processes/blob/main/Applications/4-Chutes_and_Ladders_and_MCMC/Chutes_and_Ladders_and_MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chutes and Ladders and MCMC

---

## Prompt

Original Prompt can be found [here](https://bookdown.org/kevin_davisross/applied-stochastic-processes/app-mcmc.html). A copy of the prompt along with the completed exercise can be found under [/Applications](https://github.com/NRJaggers/Applied-Stochastic-Processes/tree/main/Applications).

##Summary

This investigation concerns the boardgame Chutes and Ladders. Detailed instructions and some code have been provided in original prompt; For detailed explaination, please be sure to read the full prompt carefully.

The board has 100 spaces, labeled 1, 2, …, 100. A player starts off the board. A player generally moves on the board according to the roll of a fair six-sided die. For example, if the player is currently on space 13 and they roll a 5, then they move to space 18. However, the board also has 9 ladders which help the player climb the board and 10 chutes (slides) which knock the player back down. The game ends when the player makes it to space 100. (We’ll assume only one player.)

###Problem 1

We are interested in $T$, the number of moves (rolls) needed until spot 100 is reached (the player doesn’t need to land on 100 exactly). The position of the player after the nth move can be modeled as a Markov chain with transition matrix `Pgame` defined in the code from the prompt.

###Problem 2

Suppose you were designing a new Chutes and Ladders board. How does the placement of the chutes and ladders on the board affect the expected value of $T$? In particular, is there a way to place the chutes/ladders to minimize the expected number of moves? In this problem, you’ll write an MCMC algorithm to find the board which minimizes $E(T)$.

---

## Application

###1.

First, create the Pgame matrix to use.

####a.

Solve for $E(T)$ without first finding the distribution of $T$.

####b.

Solve for the exact distribution of $T$ and plot it. (Technically, $T$ can take infinitely many values, but feel free to cut off when the probabilities become sufficiently small.) Find $E(T)$ based on this distribution. Compare the expected value to the previous part.

####c.

Write code to run the chain and simulate the distribution of $T$. Plot the simulated distribution, use it to estimate the expected value, and compare to the previous part.

###2.

First, think about what the optimal placement might look like. Then, write an MCMC algorithm to find the board that minimizes $E(T)$. Your MCMC algorithm should involve:
- Proposing a new state, that is, proposing a new board. A board is identified by the starting spaces of the chutes and the starting spaces of the ladders (that is, the inputs to the `make_board` function).
- Finding the expected value of $T$ for the proposed board and then deciding whether or not to accept the proposed board. Note: if the proposed board is not valid (e.g., chutes/ladders land off the board), then it should be rejected.

Run the algorithm until you think it has converged and you have found the optimal board. Identify the the starting spaces for the chutes and ladders for this board.

Repeat Problem 1 for your optimal board, and compare the distribution of $T$ for your board to the one from the actual game. Write a few sentences summarizing your results.